# Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf

# pd.set_option('display.max_colwidth', 200)
# import tarfile
# import scipy.io
# from collections import defaultdict

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization

# from tensorflow.keras import activations as Ac




# from efficientnet.tfkeras import EfficientNetB3

In [3]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

In [4]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [5]:



from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import Sequential,Model
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , Input ,BatchNormalization, Concatenate, Layer , Maximum , Softmax
from tensorflow.keras.layers import  GlobalAveragePooling2D, AveragePooling2D , Conv1D , MaxPooling1D, Average, Dropout, LayerNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, GRU, ReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Read dataset

In [6]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/train.csv')
val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/val.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/test.csv')
train.head()

,filename,Classes,Labels
0,1025794.jpg,707-320,0
1,1340192.jpg,707-320,0
2,0056978.jpg,707-320,0
3,0698580.jpg,707-320,0
4,0450014.jpg,707-320,0


In [7]:
tpath = "/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/"
train_paths = train.filename.apply(lambda x: os.path.join(tpath, x))
print(train_paths.values)

# convert to one-hot-encoding-labels
train_LABELS = to_categorical(train.Labels)
train_LABELS.shape

['/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1025794.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1340192.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0056978.jpg'
 ...
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0472681.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1597829.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0197891.jpg']


(3334, 100)

In [8]:
tpath = "/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/"
val_paths = val.filename.apply(lambda x: os.path.join(tpath, x))
print(val_paths.values)

# convert to one-hot-encoding-labels
val_LABELS = to_categorical(val.Labels)
val_LABELS.shape

['/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0481847.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0810608.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1514481.jpg'
 ...
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1340345.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0765754.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1594714.jpg']


(3333, 100)

In [9]:
tpath = "/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images"
test_paths = test.filename.apply(lambda x: os.path.join(tpath, x))
print(test_paths.values)

# not convert to categorical
test_labels = test.Labels.values 
test_labels.shape

['/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1514522.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0747566.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/1008575.jpg'
 ...
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0329381.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0523192.jpg'
 '/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/images/0810303.jpg']


(3333,)

# Data preprocessing 

In [10]:
def decode_image(filename, label=None, image_size=(299, 299)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label


def augment_1(image, label=None):
    image = tf.image.random_flip_left_right(image, seed = 42)
    
    if label is None:
        return image
    else:
        return image, label

def augment_2(image, label=None):
    image = tf.image.random_contrast(image, lower=0.0, upper=1.0, seed = 42)

    if label is None:
        return image
    else:
        return image, label

def augment_3(image, label=None):
    image = tf.image.random_flip_up_down(image, seed= 42)

    if label is None:
        return image
    else:
        return image, label


def augment_4(image, label=None):
    image = tf.image.random_flip_up_down(image, seed= 42)

    if label is None:
        return image
    else:
        return image, label
    
def augment_5(image, label=None):
    image = tf.image.adjust_brightness(image, delta=0.1)

    if label is None:
        return image
    else:
        return image, label

    
def augment_6(image, label=None):
    image = tf.image.adjust_contrast(image, 2.)

    if label is None:
        return image
    else:
        return image, label


def augment_7(image, label=None):
    image = tf.image.adjust_gamma(image, gamma=1, gain=1)

    if label is None:
        return image
    else:
        return image, label

In [11]:
batch_size = 16

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_LABELS))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment_1, num_parallel_calls=AUTO)
    .map(augment_2, num_parallel_calls=AUTO)
    .map(augment_3, num_parallel_calls=AUTO)
    .map(augment_4, num_parallel_calls=AUTO)
    .map(augment_5, num_parallel_calls=AUTO)
    .map(augment_6, num_parallel_calls=AUTO)
    .map(augment_7, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

val_dataset = (tf.data.Dataset
        .from_tensor_slices((val_paths, val_LABELS))
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

test_dataset = (tf.data.Dataset
        .from_tensor_slices(test_paths)
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

# Train model

In [12]:
input_shape = (299,299,3)

inputs = Input(shape = input_shape)

# -----------------------------------------------------------------------------------------
from keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(weights='imagenet',include_top=False,input_shape= (299,299,3))
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(200,activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(100, activation = 'softmax')(x)
# -----------------------------------------------------------------------------------------
model = Model(inputs = base_model.input, outputs = output)
model.summary()

87924736/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                  

In [13]:
def Train_model(model, batch_size, EPOCHS, path):
    
    n_steps = train_LABELS.shape[0] // batch_size



    model.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),                        
                           ]
    )
    callback = ModelCheckpoint(path, 
                               monitor='val_categorical_accuracy', 
                               verbose=1, 
                               save_best_only=True,
                               mode='max')
    
    history_1 = model.fit(train_dataset ,
                          steps_per_epoch = n_steps,  
                          epochs=  EPOCHS, 
                          batch_size = batch_size , 
                          validation_data = val_dataset,
                          callbacks = [callback]
                        )
    
    return Model

In [14]:
# define parameters 
path = "/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/train_test_val/InceptionV3.hdf5"
batch_size = 16
EPOCHS = 60

In [20]:

print('Training')
model = Train_model(model, batch_size, EPOCHS,path)


In [16]:
# print("best model: ")
# model.load_weights("/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/train_test_val/model_cont.hdf5")
# model.compile(
#                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, decay=0.0),
#                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
#                 metrics = [
#                         keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),                        
#                         ]
# )


# n_steps = train_LABELS.shape[0] // batch_size

# f1_callback_concat = ModelCheckpoint("/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/train_test_val/model_cont_1.hdf5", 
#                             monitor='val_categorical_accuracy', 
#                             verbose=1, 
#                             save_best_only=True,
#                             mode='max')


# history_1 = model.fit(  train_dataset , 
#                         steps_per_epoch = n_steps,  
#                         epochs= EPOCHS, 
#                         batch_size = batch_size , 
#                         validation_data = val_dataset,
#                         callbacks = [f1_callback_concat]
#                         )


# Evaluate model

In [17]:
path = "/content/drive/MyDrive/Colab Notebooks/Aircraft/fgvc-aircraft-2013b/data/train_test_val/InceptionV3.hdf5"

In [18]:
print("best model ")
model.load_weights(path)
model.compile(
                optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, decay=0.0),
                loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                metrics = 
                        [keras.metrics.CategoricalAccuracy(name='categorical_accuracy')]
)

best model 


In [19]:
pred = model.predict(val_dataset)
print("Accuracy on Validation Set : ",accuracy_score(val.Labels.values, np.argmax(pred,axis = 1)))

Accuracy on Validation Set :  0.7728772877287728
